In [97]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import ast

ds = pd.read_csv("perfect_datasets/perfect_dataset_v3.csv")
ds_overview = pd.read_csv("perfect_datasets/perfect_dataset_v1.csv")

cv = CountVectorizer(max_features=1000, stop_words="english")

vectors = cv.fit_transform(ds["tags"]).toarray()

similarity = cosine_similarity(vectors)

temp = pd.DataFrame()
temp["title"] = ds["title"].str.lower().str.split()
temp["title"] = temp["title"].apply(lambda x: " ".join(x))
temp["title"] = temp["title"].apply(lambda x: x.replace(":", ""))

movie_sugg = []

c = 0

def recommend(movie):
    global c
    # if(ds[(ds["title"].str.lower()).str.startswith(movie.lower())].empty):
    if(movie == ""):
        print("Enter a movie name")
        return
    elif(ds[ds["title"].str.lower() == movie.lower()].empty):
        for i in temp["title"]:
            if(movie.lower() in i) and c==0:
                movie_sugg.append(ds.iloc[temp[temp["title"] == i].index[0]].title)
        print("did you mean: ", movie_sugg, end="\n\n") if c==0 else print("No movie found")
        c = 1
        return
    # index = ds[(ds["title"].str.lower()).str.startswith(movie.lower())].index[0]
    index = ds[(ds["title"].str.lower()) == movie.lower()].index[0]
    # distance = similarity.iloc[index,:]
    distance = similarity[index]
    movie_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:11]

    for i in movie_list:
        print("Movie Id: " + str(ds.iloc[i[0]].movie_id))
        print("Movie Name: " + ds.iloc[i[0]].title)
        print("Movie Name: " + ds_overview.iloc[i[0]].overview, end="\n\n")
    
inp = str(input("Enter the movie name: "))
recommend(inp)

if movie_sugg != []:
        movie_re_input = str(input("Enter the movie name: "))
        recommend(movie_re_input)

did you mean:  ['The Hobbit: The Battle of the Five Armies', 'The Hobbit: The Desolation of Smaug', 'The Hobbit: An Unexpected Journey']

Movie Id: 49051
Movie Name: The Hobbit: An Unexpected Journey
Movie Name: Bilbo Baggins, a hobbit enjoying his quiet life, is swept into an epic quest by Gandalf the Grey and thirteen dwarves who seek to reclaim their mountain home from Smaug, the dragon.

Movie Id: 57158
Movie Name: The Hobbit: The Desolation of Smaug
Movie Name: The Dwarves, Bilbo and Gandalf have successfully escaped the Misty Mountains, and Bilbo has gained the One Ring. They all continue their journey to get their gold back from the Dragon, Smaug.

Movie Id: 766
Movie Name: Army of Darkness
Movie Name: A man is accidentally transported to 1300 A.D., where he must battle an army of the dead and retrieve the Necronomicon so he can return home.

Movie Id: 1271
Movie Name: 300
Movie Name: Based on Frank Miller's graphic novel, "300" is very loosely based the 480 B.C. Battle of Therm